In [102]:
import pandas as pd
import re

In [103]:
original_df = pd.read_csv(r'C:\Users\acoullard\Programming\Python\NEU_course_transfer_web_scraper\results\2-21-25_Scrape.csv')
fice_table = pd.read_csv(r'C:\Users\acoullard\Programming\Python\NEU_course_transfer_web_scraper\results\fice_table.csv')

In [104]:
# extract and store all extra courses from records that have multiple courses

multi_courses = original_df[original_df['NEUCourse'].str.contains(" and [A-Z]{2,}\s*\d{4}\s-.*", regex=True)]
match_list = []

for i in multi_courses.index:
    row = original_df.loc[i].copy()
    course_names_str = original_df.at[i, 'NEUCourse']
    matches = re.findall(r"and [A-Z]{2,}\s*\d{4}\s-.*?(?=and [A-Z]{2,}\s*\d{4}\s-.*|\Z)", course_names_str)
    matches = [match.strip()[4:] for match in matches]
    for match in matches:
        row["NEUCourse"] = match
        # print(row)
        match_list.append(row)
multi_courses_df = pd.DataFrame(match_list)
# multi_courses_df

In [105]:
# stip away multi courses that were already extracted before

def remove_multi_courses(row):
    row['NEUCourse'] = re.sub(r"and [A-Z]{2,}\s*\d{4}\s-.*\Z", "", row['NEUCourse']).strip()
    return row
multi_courses_removed = original_df.apply(remove_multi_courses, axis=1)
# multi_courses_removed

In [106]:
combined = pd.concat([multi_courses_removed, multi_courses_df])
# combined

In [107]:
valid_courses = combined[combined['NEUCourse'].str.contains(r'[A-Z]+\s*\d{4}', regex=True)]
# valid_courses
# print(valid_courses['NEUCourse'].unique())

In [108]:
# Extract codes using vectorized string operations
codes_out = valid_courses.copy()
codes_out['NEUCode'] = (valid_courses['NEUCourse'].str.extract(r'([A-Z]+\s*\d{4})'))
codes_out['NEUCode'] = codes_out['NEUCode'].str.replace(r'\s+', '', regex=True)
codes_out

,NEUCourse,TransferCourse,EffectiveDates,NUCore,NUPath,FICE,NEUCode
0,ARCH3155 - Studio Abroad,ARH 255,NaN,NaN,NaN,007531,ARCH3155
1,ARCH1990 - Elective,LA 219,NaN,NaN,NaN,007531,ARCH1990
2,BIOL1117 -,BIO 204,NaN,Science/Technology 1,NUpath Analyzing/Using Data,002666,BIOL1117
3,CHEM1211 - General Chemistry 1,CHE 111,NaN,Science/Technology 1,NUpath Natural/Designed World,002666,CHEM1211
4,CHEM1214 - General Chemistry 2,CHE 112,NaN,NaN,NUpath Natural/Designed World,002666,CHEM1214
...,...,...,...,...,...,...,...
62091,ECON1116 - Principles of Microeconomics,Introduction to Economics,NaN,Social Sciences 1,NUpath Analyzing/Using Data,F00034,ECON1116
62152,BIOL2218 - Lab for BIOL 2217,BIO210,NaN,Science/Technology 1,NUpath Natural/Designed World,003996,BIOL2218
62153,BIOL2220 - Lab for BIOL 2219,BIO211,NaN,NaN,NUpath Analyzing/Using Data,003996,BIOL2220
62154,BIOL2222 - Lab for BIOL 2221,BIO225,NaN,Science/Technology 1,NaN,003996,BIOL2222
